## Initial data solver using relaxation

We try to solve the time symmetric initial data when given a $ρ$ field on a cube. 
We put Robin boudary conditions.

Thus we shall be solving:

$$
ψ_{tt} = \Delta ψ ψ^5 ρ \;\;\;\; \vec{x}\cdot \nabla ψ + (1-ψ) = 0
$$

Using second order finite differences inside the domain and first order at the boudaries we get for the boundary values (which are not evolved):

For boundary conditions we shall put an outgoing wave, but taking into account that we want a radial wave and that something which at first order should be a monopolar field.

$$
ψ_t = (x^i\partial_i ψ - (ψ - ψ_0))/r
$$ 
Same for the time derivative, but that part should be unnessary becuse this should decay to a static solution.

The solution we are seeking is of the form of an outgoing wave plus a static solution, the one we are after. It well behave as a monopole at large distance, plus a constant.

$$
\psi(t,x) =\frac{f(t-r) + M}{r} + \psi_0
$$

Thus we have, 
$$
\partial_t \psi = \frac{f'(t-r)}{r}, \;\;\;\;\; \frac{\vec{x}\cdot \nabla \psi}{r} = -f'/r - f/r^2 - M/r^2 
$$

therefore asymtotically we should have, 
$$ 
\partial_t \psi = - \frac{\vec{x}\cdot \nabla \psi}{r} - (f+M)/r^2 = - \frac{\vec{x}\cdot ∇\psi}{r} - (\psi - \psi_0)/r 
$$

In [1]:
using SummationByPartsOperators
using Plots
using Revise
using Distributions
using FFTW
using LinearAlgebra
using Base.Threads
using HDF5
 
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")


In [2]:
J = (45,45,45)
Box_x = (-22.0, 22.0, -22.0, 22.0, -22.0, 22.0)

Dx = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
Dy = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
Dz = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
D2x = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
D2y = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
D2z = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
x = collect(SummationByPartsOperators.grid(Dx))
y = collect(SummationByPartsOperators.grid(Dy))
z = collect(SummationByPartsOperators.grid(Dz));



In [ ]:
dx = differentials(Box_x, J, periodic=false)

In [4]:
n_fields = 2 # the field and its derivative 
u = zeros(n_fields,J...)
du = similar(u)
d2 = zeros(J...);
dxu = similar(u);
dyu = similar(u);
dzu = similar(u);

In [ ]:

x0 = [0.0,0.0,0.0]
r0 = 4.0 # 10.0
p = 4 # para fuente chichon
p = 1 # para fuente carlos (A0)
par = (x,y,z,x0,Box_x, r0, p, J)

#ρ = zeros(J...)

#ρ =  get_source(chichon, par);
ρ =  get_source(carlos, par);
heatmap(ρ[:,:,20],aspectratio=1)
ϕ_ex = get_source(carlossol,par)
heatmap(ϕ_ex[:,:,20],aspectratio=1)

#@show norm(ρ)
#maximum(abs.(ρ))

In [154]:
T = 30.0
M = 750
dt = T/M
t = 0.0

#null initial data

for i in 1:J[1]
    for j in 1:J[2]
        for k in 1:J[3]
            r = sqrt(x[i]^2 + y[j]^2 + z[k]^2)
            u[1,i,j,k] = 1.0 #+ exp(-r^2/4.0)
            u[2,i,j,k] = 0.0
        end
    end
end

#norm(D2x*u[1,:,20,20])
#plot(D2x*u[1,:,20,20])
#plot!(u[1,:,20,20].-1)

In [ ]:

v = zeros(2,M+1,J...)
#u[:,:,:,:] .= v[:,501,:,:,:]

τ = 0.0 #damping must be negative
par = (1.0, 1.0, τ) # (a, b, τ)
par_F = (x,y,z,dxu,dyu,dzu,Dx,Dy,Dz,D2x,D2y,D2z,d2,dx,ρ,J, par)

norms = zeros(2,M+1)

v[:,1,:,:,:] = u[:,:,:,:]

for m in 2:M+1
    RK4_Step!(FC,u,t,dt,par_F)
    t = t + dt
    norms[:,m] = [norm(u[1,:,:,:]),norm(u[2,:,:,:])]
    if mod(m+1,10)==0
        println("t = $t, norm u_1 = $(norms[1,m]) norm u_2 = $(norms[2,m])")
    end
    v[:,m,:,:,:] = u[:,:,:,:]
end
    
norm(d2)

In [ ]:
heatmap(v[1,101,:,:,(J[3]+1)÷2] - 0.0*v[1,1,:,:,(J[3]+1)÷2] - 0.0*ϕ_ex[:,:,(J[3]+1)÷2],aspectratio=1)

In [ ]:
plot(v[1,751,:,(J[2]+1)÷2,(J[3]+1)÷2] - ϕ_ex[:,(J[2]+1)÷2,(J[3]+1)÷2],aspectratio=1)

In [ ]:
anim_plot = @animate for i in 1:10:M+1
    plot(v[1,i,:,(J[2]+1)÷2,(J[3]+1)÷2] - ϕ_ex[:,(J[2]+1)÷2,(J[3]+1)÷2], ylim=(0.0,3.0))
end


In [ ]:

gif(anim_plot, "carlos_phi_t_plot.gif", fps = 10)

In [ ]:
anim = @animate for i ∈ 1:10:M+1
    surface(v[2,i,:,:,(J[3]+1)÷2].-1.0,aspectratio=1)
end

gif(anim, "carlos_phi_t.gif", fps = 10)
#heatmap(v[2,1,:,:,(J[3]+1)÷2].-0.0,aspectratio=1)

In [ ]:
plot(norms[1,:], label="phi_norm")
plot!(norms[2,:], label="phi_t_norm")
#png("convergence_u5_T$(T)_k$(τ)")

In [ ]:
surface(x,y,u[1,:,:,(J[3]+1)÷2])

In [ ]:
plot(x,u[1,:,(J[2]+1)÷2,(J[3]+1)÷2])

In [ ]:
plot(x,u[2,:,(J[2]+1)÷2,(J[3]+1)÷2])

In [ ]:
norm(u)/prod(J)

In [ ]:
argmax(abs.(u))

In [ ]:
plot(u[2,2,43,:])

In [148]:

l = FC(v[:,751,:,:,:],1.0,par_F);


In [ ]:
l[2,(J[1]+1)÷2,(J[2]+1)÷2,(J[3]+1)÷2]